In [240]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [241]:
%%javascript

// require(["https://unpkg.com/higlass-multivec/dist/higlass-multivec"],
//     function(hglib) {

// });

require(["http://localhost:8082/higlass-multivec.js"],
    function(hglib) {

});

<IPython.core.display.Javascript object>

In [242]:
%%javascript

require(["https://unpkg.com/higlass-pileup/dist/higlass-pileup.min.js"],
    function(hglib) {

});

<IPython.core.display.Javascript object>

In [318]:
import higlass
from higlass.tilesets import multivec, bam, Tileset
from higlass.client import Track, View

# ts = multivec("../data/t2t/chm13.draft_v1.0.hifi.bam.mv5")
# bam_ts = bam(
#     'https://s3.amazonaws.com/nanopore-human-wgs/chm13/assemblies/alignments/chm13.draft_v1.0.hifi.bam'
# )

ts = multivec("../data/t2t/chm13.draft_v1.0.pcrfree.bam.mv5")
bam_ts = bam(
    'https://s3.amazonaws.com/nanopore-human-wgs/chm13/assemblies/alignments/chm13.draft_v1.0.pcrfree.bam'
)

display, server, viewconf = higlass.display(
    [View([
        Track('top-axis', height=20),
        Track('chromosome-labels', server="https://resgen.io/api/v1",
              tilesetUid="P5Hodo9FS--bLAai7Ibjrg"),
        Track(track_type="horizontal-stacked-bar",
              position='top', tileset=ts, height=100,
              options={"colorScale": colorScale, "valueScaling": "log"} ),
        Track(track_type="pileup", position="top", tileset=bam_ts, height=200)
    ], initialXDomain=[744180000, 744260000])]
#     ], initialXDomain=[692477935.3862978, 692547772.7935884])]
)
display

# regions: 2110500000
# regions: 2216940000
# regions: 2320000000
# regions: 2810000000
# regions: 2856000000
# regions: 744214655 (solitary deletion)

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'HnyLYcamS1-3pirm7RYKHw', 'tracks': {'top': [{'ty…

In [320]:
f = h5py.File("../data/t2t/chm13.draft_v1.0.hifi.bam.mv5", 'r+')

row_infos = [
    {'name': 'M', 'color': '#DCDCDC'},  #M
    {'name': 'A', 'color': '#08519c'},  #A
    {'name': 'C', 'color': '#fe9929'},  #C
    {'name': 'G', 'color': '#993404'},  #G
    {'name': 'T', 'color': '#6baed6'},  #T
    {'name': 'S', 'color': '#808080'},  #S
    {'name': 'I', 'color': 'rgba(255, 0, 255, 0.5)'},      #I
    {'name': 'D', 'color': 'rgba(255, 128, 128, 0.5)'},      #D
    {'name': 'H', 'color': 'rgba(40, 40, 40, 0.5)'},      #H
    {'name': 'N', 'color': 'rgba(10, 10, 10, 0.5)'}      #N
]

import json
f['info'].create_dataset('row_infos', data=json.dumps(row_infos))
print(f['info'])

ValueError: Unable to create dataset (name already exists)

In [310]:
f.close()

In [260]:
def min_agg(x):
    return np.min(x.T.reshape((x.shape[1], -1, 2)), axis=2).T

def max_agg(x):
    return np.max(x.T.reshape((x.shape[1], -1, 2)), axis=2).T

def min_max_agg(x, min_cols, max_cols, combine):
    mins = min_agg(x[:,min_cols])
    maxs = max_agg(x[:,max_cols])
    
    new_arr = np.array([[]])
    
    new_arr = np.hstack(combine(mins, maxs))
    return new_arr
    
arr = np.array([[1,2,3],[4,5,6],[14,8,9],[10,11,12]])

min_max_agg(arr, min_cols=[1], max_cols=[0,2],
            combine=lambda mins, maxs: [maxs[:,[0]], mins[:,[0]], maxs[:,[1]]])

all_cols = list(range(10))

bam_min_max_agg = lambda arr: min_max_agg(
    arr, min_cols=[0], max_cols=all_cols[1:],
    combine=lambda mins, maxs: [mins[:,[0]], maxs[:,all_cols[:-1]]]
)

In [293]:
def reads_to_array(f_in, h_out, ref, chrom_len, start=None, end=None):
    """Convert BAM file reads to an HDF5 array.

    Arguments:

    f_in: The pysam AlignmentFile handle
    h_out: An hdf5 file handle to store the output arrays
    ref: The chromosome name
    chrom_len: The length of the chromosome

    """
    logger.info("Creating array for chrom: %s with length: %d", ref, chrom_len)
    if not start:
        start = 0
    if not end:
        end = chrom_len
        
    reads = f_in.fetch(ref, start, end)

    arr = np.zeros((10, chrom_len))

    subs = {
        "M": 0,
        "A": 1,
        "C": 2,
        "G": 3,
        "T": 4,
        "S": 5,
        "I": 6,
        "D": 7,
        "H": 8,
        "N": 9
    }

    logger.info("Finished allocating arrays")
    counter = 0

    for read in reads:
        counter += 1

        if counter % 5000 == 0:
            logger.info("Processed %d reads", counter)

        if read.reference_start is None or read.reference_end is None:
            continue
            
        if read.query_sequence:
            try:
                ap = [
                    p
                    for p in read.get_aligned_pairs(with_seq=True, matches_only=True)
                    if p[2].islower()
                ]

                for p in ap:
                    arr[subs["M"]][p[1] + 1] -= 1
                    arr[subs[read.query_sequence[p[0]]]][p[1] + 1] += 1
            except ValueError:
                continue

        #     print("read", read.reference_start)
        arr[subs["M"]][read.reference_start + 1 : read.reference_end + 1] += 1

        for start, op, oplen in get_cigar_substitutions(read):
            if op == "I":
                arr[subs[op]][start + 1] += 1
            else:
                arr[subs[op]][start + 1 : start + 1 + oplen] += 1
            
            if op == 'D':
                arr[subs['M']][start + 1 : start + 1 + oplen] -= 1



    logger.info("Dumping array with shape: %s", str(arr.T.shape))

    h_out.create_dataset(ref, data=arr.T, compression="gzip")
    pass

In [301]:
%%time

# filename = '../data/t2t/chm13.draft_v1.0.hifi.bam'
# indexfile = '../data/t2t/chm13.draft_v1.0.hifi.bam.bai'
output_file = '../data/t2t/out.mv5'
filename = '../data/t2t/small.bam'
indexfile = '../data/t2t/small.bam.bai'

f = pysam.AlignmentFile(filename, index_filename=indexfile)

from tempfile import TemporaryDirectory
import h5py
import os.path as op

import logging

logger = logging.getLogger(__name__)
import clodius.multivec as cmv

# region = [52_023_000, 52_024_000]
region = [0, 200_000]
references = ['chr1']
# region = [8_000_000, 11_000_000]
lengths = [region[1] + 100_000]

start = region[0]
end=region[1]

# references = ['chr1']
# lengths = [200_000]

# ref = 'chr1'
# start = 0
# end=1000

row_infos = [
    {'name': 'M', 'color': '#DCDCDC'},  #M
    {'name': 'A', 'color': '#08519c'},  #A
    {'name': 'C', 'color': '#fe9929'},  #C
    {'name': 'G', 'color': '#993404'},  #G
    {'name': 'T', 'color': '#6baed6'},  #T
    {'name': 'S', 'color': '#808080'},  #S
    {'name': 'I', 'color': 'rgba(255, 0, 255, 0.5)'},      #I
    {'name': 'D', 'color': 'rgba(255, 128, 128, 0.5)'},      #D
    {'name': 'H', 'color': 'rgba(40, 40, 40, 0.5)'},      #H
    {'name': 'N', 'color': 'rgba(10, 10, 10, 0.5)'}      #N
]


with TemporaryDirectory() as tmp_dir:
    h_mid = h5py.File(op.join(tmp_dir, "mid.h5"), "w")

    reads_to_array(f, h_mid, references[0], lengths[0], start=start, end=end)

    logger.info("Creating multivec array")
    cmv.create_multivec_multires(
        h_mid,
        zip(references, lengths),
#         agg=sum_agg,
#         agg=min_agg,
        agg=my_min_max_agg,
        #     agg=log_sum_exp_agg,
        starting_resolution=1,
        row_infos=row_infos,
        output_file=output_file,
        tile_size=256,
    )

CPU times: user 771 ms, sys: 68.5 ms, total: 840 ms
Wall time: 846 ms


In [302]:
import higlass
from higlass.tilesets import multivec, bam
from higlass.client import Track, View

import pandas as pd

def chromsizes_array_to_series(chromsizes):
    """
    Convert an array of [[chrname, size]...] values to a series
    indexed by chrname with size values
    """
    print('chromsizes', list(chromsizes))
    
    chrnames = [c[0] for c in chromsizes]
    chrvalues = [c[1] for c in chromsizes]

    print(chrnames)
    print(chrvalues)
    return pd.Series(np.array([int(c) for c in chrvalues]), index=chrnames)

chromsizes = chromsizes_array_to_series(list(zip(references, lengths)))
ts = multivec("../data/t2t/out.mv5")
bam_ts = bam(filename,
            chromsizes=chromsizes)

display, server, viewconf = higlass.display(
    [View([
        Track('top-axis', height=20),
        Track(track_type="horizontal-stacked-bar",
              position='top', tileset=ts, height=100,
              options={"valueScaling": "log"} ),
        Track(track_type="pileup", position="top", tileset=bam_ts, height=200)
    ], initialXDomain=region)]
)
display

# regions: 2110500000
# regions: 2216940000
# regions: 2320000000
# regions: 2810000000
# regions: 2856000000

chromsizes [('chr1', 300000)]
['chr1']
[300000]


HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'YIK01R9PQYKMLEOyzmDZZg', 'tracks': {'top': [{'ty…

In [238]:
%%time


reads_to_array(f, h_mid, ref, lengths[0], start=52_030_000, end=end)


ValueError: invalid coordinates: start (52030000) > stop (20000000)

In [263]:
import higlass
from higlass.client import Track, View

display, server, viewconf = higlass.display(
    [View([
        Track('top-axis', height=20),
        Track(track_type="horizontal-stacked-bar", position='top', tileset=ts, height=50,
              options={"colorScale": colorScale, "valueScaling": "log"} ),
        Track(track_type="pileup", position='top', tileset=bam_ts, height=200 )
    ])]
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'cLq8ZK7USlmjujAWBi_nFA', 'tracks': {'top': [{'ty…

In [113]:
from higlass.tilesets import Tileset, bam

# def bam(filepath, index_filename, uuid=None, **kwargs):
#     from clodius.tiles.bam import tileset_info, tiles

#     return Tileset(
#         uuid=uuid,
#         tileset_info=lambda: tileset_info(filepath),
#         tiles=lambda tile_ids: tiles(filepath, tile_ids, index_filename=index_filename),
#         **kwargs
#     )

bam_ts = bam(filename,indexfile)

In [114]:
import higlass
from higlass.tilesets import Tileset, bam
from higlass.client import Track, View

bam_ts = bam(filename,indexfile)

display, server, viewconf = higlass.display(
    [View([
        Track('top-axis', height=20),
        Track(track_type="pileup", position='top', tileset=bam_ts, height=200 )
    ], initialXDomain = [
        0,
        2000
      ])]
)
display

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'KSNHPIFhRbWkKUWm3v0CAw', 'tracks': {'top': [{'ty…

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'AZ5AQQ6xS_iq3GH7drwBHQ', 'tracks': {'top': [{'ty…